In [ ]:
!pip install cartopy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import  xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import math
from tqdm.auto import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

from cartopy.util import add_cyclic_point
from matplotlib import animation
from tqdm.auto import tqdm
from datetime import datetime,timedelta

from matplotlib.cm import get_cmap
warnings.filterwarnings("ignore")

In [ ]:
ds=xr.open_dataset('/content/drive/MyDrive/AP_HW/ERA5 Data/total_cloud_cover/tcc-2024.nc')
ds


*   We calculated the **total cloud cover anomalies** for each event using the dates from `HW_list_rolling_def.csv` for **pre-monsoon (MAM)** and **monsoon (JJAS)** seasons.  
*   The **climatology** for **pre-monsoon (MAM)** and **monsoon (JJAS)** season was derived from ERA5 total cloud cover data (1981–2010) and computed in :
    - `climatology_calculation_total_colud_cover.ipynb`




# MAM

In [ ]:
# @title Heat date list
list_hw=pd.read_csv('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/HW_list_rolling_def.csv')
list_hw['Starting_Date'] = pd.to_datetime(list_hw["Starting_Date"])
list_hw['Ending_Date'] = pd.to_datetime(list_hw["Ending_Date"])

# separate hw for mam season
mam_hw_list =list_hw[(list_hw['Starting_Date'].dt.month >= 3) & (list_hw['Starting_Date'].dt.month <= 5)]
mam_hw_list.reset_index(inplace=True,drop=True)
mam_hw_list

In [ ]:
# @title Event data for composite analysis-anomaly

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as colors

anom_list=[]  # List to store anomalies for each event

for i in tqdm(range(len(mam_hw_list['Starting_Date']))):
    # Extract event info
    year = mam_hw_list['Starting_Date'].dt.year.iloc[i]
    start_time = str(mam_hw_list['Starting_Date'][i])
    end_time = str(mam_hw_list['Ending_Date'][i])

    # Adjust times to Bangladesh timezone
    start_time = str(np.datetime64(start_time) - np.timedelta64(6, 'h'))
    end_time = str(np.datetime64(end_time) - np.timedelta64(6, 'h') + np.timedelta64(24, 'h'))
    time_range = slice(start_time, end_time)
    event_no = i + 1

    ###################################
    ############# tcc ################
    # Load total cloud cover data and climatology files
    ds_tcc = xr.open_dataset(f"/content/drive/MyDrive/AP_HW/ERA5 Data/total_cloud_cover/tcc-{year}.nc")
    tcc_clim = xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/tcc/tcc_climatology-mam.nc')

    # Select event period and compute mean
    ds_tcc = ds_tcc.sel(valid_time=time_range)
    ds_tcc = ds_tcc.mean(dim='valid_time', skipna=True)

    # Compute anomaly and assign event number
    tcc_anom = ds_tcc - tcc_clim
    tcc_anom = tcc_anom.assign_coords(event_no=event_no)

    # Add to anomaly list
    anom_list.append(tcc_anom)

# Concatenate all event anomalies and save to file
tcc_events_anom_mam = xr.concat(anom_list, dim='event_no')
tcc_events_anom_mam.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/tcc/data/tcc_events_anom-mam.nc')


# JJAS

In [ ]:
# @title Heat date list
list_hw=pd.read_csv('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/HW_list_rolling_def.csv')
list_hw['Starting_Date'] = pd.to_datetime(list_hw["Starting_Date"])
list_hw['Ending_Date'] = pd.to_datetime(list_hw["Ending_Date"])

# separate hw for jjas season
jjas_hw_list =list_hw[(list_hw['Starting_Date'].dt.month >= 6) & (list_hw['Starting_Date'].dt.month <= 9)]
jjas_hw_list.reset_index(inplace=True,drop=True)
jjas_hw_list

In [ ]:
# @title Event data for composite analysis-anomaly

import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
import numpy as np
import matplotlib.colors as colors

anom_list = []  # List to store anomalies for each event

for i in tqdm(range(len(jjas_hw_list['Starting_Date']))):
    # Extract event info
    year = jjas_hw_list['Starting_Date'].dt.year.iloc[i]
    start_time = str(jjas_hw_list['Starting_Date'][i])
    end_time = str(jjas_hw_list['Ending_Date'][i])

    # Adjust times to Bangladesh timezone
    start_time = str(np.datetime64(start_time) - np.timedelta64(6, 'h'))  # Subtract 6 hrs
    end_time = str(np.datetime64(end_time) - np.timedelta64(6, 'h') + np.timedelta64(24, 'h'))  # Include end date
    time_range = slice(start_time, end_time)
    event_no = i + 1

    ###################################
    ############# tcc ################
    # Load total cloud cover data and climatology files
    ds_tcc = xr.open_dataset(f"/content/drive/MyDrive/AP_HW/ERA5 Data/total_cloud_cover/tcc-{year}.nc")
    tcc_clim = xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/tcc/tcc_climatology-jjas.nc')

    # Select event period and compute mean
    ds_tcc = ds_tcc.sel(valid_time=time_range)
    ds_tcc = ds_tcc.mean(dim='valid_time', skipna=True)

    # Compute anomaly and assign event number
    tcc_anom = ds_tcc - tcc_clim
    tcc_anom = tcc_anom.assign_coords(event_no=event_no)

    # Add to anomaly list
    anom_list.append(tcc_anom)

# Concatenate all event anomalies and save to file
tcc_events_anom_jjas = xr.concat(anom_list, dim='event_no')
tcc_events_anom_jjas.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/tcc/data/tcc_events_anom-jjas.nc')


In [ ]:
print('done')